# Supervised Learning: from scratch and with scikit-learn

Examples showing the math behind basic algorithms and their scikit-learn counterparts.

In [ ]:
!pip install numpy scikit-learn matplotlib -q

Scikit-learn is a powerful library for machine learning in Python, providing efficient tools for data mining and data analysis. It is built on NumPy, SciPy, and matplotlib.

Documentation: [scikit-learn documentation](https://scikit-learn.org/stable/)

In [ ]:
import numpy as np
from sklearn.datasets import make_regression, make_classification, load_digits, load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Perceptron, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

This cell imports `numpy` for numerical arrays, utilities from `scikit-learn` for datasets and models, and `matplotlib` for plotting.

## 0. Linear regression from scratch

Linear regression seeks to model the relationship between a set of input features $x \in \mathbb{R}^d$ and a target variable $y \in \mathbb{R}$ by fitting a linear function:

$$
y = w^T x + b
$$

where $w \in \mathbb{R}^d$ is the vector of weights (coefficients) and $b \in \mathbb{R}$ is the intercept (bias).

Given a dataset of $n$ samples $\{(x^{(i)}, y^{(i)})\}_{i=1}^n$, we can write the model for all samples in matrix form. Let $X \in \mathbb{R}^{n \times d}$ be the matrix whose rows are the input vectors $x^{(i)}$, and $y \in \mathbb{R}^n$ be the vector of targets.

To include the intercept $b$ in the model, we augment $X$ with a column of ones:

$$
X_b = \begin{bmatrix} 1 & x_1^{(1)} & \cdots & x_d^{(1)} \\
1 & x_1^{(2)} & \cdots & x_d^{(2)} \\
\vdots & \vdots & & \vdots \\
1 & x_1^{(n)} & \cdots & x_d^{(n)} \end{bmatrix} \in \mathbb{R}^{n \times (d+1)}
$$

and define the parameter vector

$$
\theta = \begin{bmatrix} b \\ w_1 \\ \vdots \\ w_d \end{bmatrix} \in \mathbb{R}^{d+1}
$$

The model becomes:

$$
y \approx X_b \theta
$$

The goal is to find $\theta$ that minimizes the sum of squared errors (the least-squares criterion):

$$
L(\theta) = \|y - X_b \theta\|^2 = (y - X_b \theta)^T (y - X_b \theta)
$$

To find the minimum, we set the gradient with respect to $\theta$ to zero:

$$
\frac{\partial L}{\partial \theta} = -2 X_b^T (y - X_b \theta) = 0
$$

Solving for $\theta$ gives the closed-form solution:

$$
X_b^T X_b \theta = X_b^T y \\
\implies \theta = (X_b^T X_b)^{-1} X_b^T y
$$

This formula computes the optimal weights and intercept that best fit the data in the least-squares sense, assuming $X_b^T X_b$ is invertible. In summary, linear regression finds the hyperplane in $\mathbb{R}^d$ that minimizes the squared distance to all data points.

In [ ]:
X, y, true_coef = make_regression(n_samples=100, n_features=1, noise=10.0, coef=True, random_state=42, bias=0)
print("True coefficients:", true_coef)
# Plot the data
plt.scatter(X, y, color="blue", label="data")
plt.xlabel("X")
plt.ylabel("y")
plt.title("Linear Regression Data")
plt.legend()
plt.show()

In [ ]:
X_b = np.c_[np.ones((len(X),1)), X]
theta = np.linalg.inv(X_b.T @ X_b) @ X_b.T @ y
print("True coef:", true_coef)
print("Closed-form coef:", theta[1])
print("Intercept:", theta[0])

The coefficients above are computed using the closed-form equation.

In [ ]:
x_grid = np.linspace(X.min(), X.max(), 100).reshape(-1,1)
y_pred = theta[1]*x_grid + theta[0]

plt.scatter(X, y, color="blue", label="data")
plt.plot(x_grid, y_pred, color="red", label="fit")
plt.legend()
plt.tight_layout()
plt.show()

### scikit-learn linear regression

`LinearRegression` in scikit-learn solves the same least-squares problem for us.

In [ ]:
model = LinearRegression()
model.fit(X, y)
print("sklearn coef:", model.coef_[0])
print("sklearn intercept:", model.intercept_)

## 1. k-NN classification from scratch

k-Nearest Neighbors (k-NN) is a simple, intuitive algorithm for classification and regression. For a new sample, k-NN finds the $k$ closest points in the training data—using a distance metric such as Euclidean distance:

$$
d(x_i, x_j) = \sqrt{\sum_{k=1}^d (x_{i,k} - x_{j,k})^2}
$$

and assigns the most common class (for classification) or the average value (for regression) among those neighbors.

The k-NN process:

1. **Choose $k$:** Number of neighbors.
2. **Compute distances:** Measure distance from the test sample to all training samples.
3. **Find neighbors:** Select the $k$ closest training samples.
4. **Predict:** Use majority vote (classification) or average (regression).

k-NN is non-parametric and instance-based, relying on the entire training set for predictions. The choice of $k$ and distance metric affects performance: small $k$ can be sensitive to noise, while large $k$ may smooth out class boundaries.


In [ ]:
X, y = load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
def knn_predict(X_train, y_train, X_test, k=3):
    preds = []
    for x in X_test:
        dists = np.linalg.norm(X_train - x, axis=1)
        idx = np.argsort(dists)[:k]
        preds.append(np.bincount(y_train[idx]).argmax())
    return np.array(preds)

In [ ]:
preds = knn_predict(X_train, y_train, X_test, k=3)
print("Accuracy:", accuracy_score(y_test, preds))

### scikit-learn k-NN

`KNeighborsClassifier` performs this neighbor search efficiently.

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
sk_preds = knn.predict(X_test)
print("Accuracy:", accuracy_score(y_test, sk_preds))

## 2. Perceptron with gradient descent

The perceptron algorithm is an iterative method for finding a linear decision boundary between two classes. The steps are:

1. **Initialize** the weights $w$ and bias $b$ to zero (or small random values).
2. **For each training sample** $(x_i, y_i)$, where $y_i \in \{-1, 1\}$:
    - Compute the margin: $m_i = y_i (w^T x_i + b)$.
    - If $m_i < 0$ (i.e., the sample is misclassified):
      - Update the weights: $w \leftarrow w + \eta y_i x_i$
      - Update the bias: $b \leftarrow b + \eta y_i$
    - Here, $\eta$ is the learning rate (often set to 1).
3. **Repeat** over the dataset for a fixed number of epochs or until all samples are correctly classified.

The perceptron only updates its parameters when it makes a mistake. This process continues until the algorithm converges (if the data is linearly separable) or the maximum number of iterations is reached.

In [ ]:
X, y = make_classification(n_samples=200, n_features=2, n_informative=2, n_redundant=0, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
y_train_signed = np.where(y_train==0, -1, 1)

In [ ]:
w = np.random.randn(X_train.shape[1])
b = 0.
learning_rate = 0.1
num_iterations = 100

In [ ]:
for _ in range(num_iterations):
    
    margins = y_train_signed * (X_train @ w + b)
    mask = margins < 0

    if not mask.any():
        break
    
    grad_w = -(y_train_signed[mask,None] * X_train[mask]).mean(axis=0)
    grad_b = -(y_train_signed[mask]).mean()
    
    w -= learning_rate * grad_w
    b -= learning_rate * grad_b

preds = (X_test @ w + b >= 0).astype(int)
print("Accuracy:", accuracy_score(y_test, preds))

### scikit-learn Perceptron

scikit-learn's `Perceptron` implements the same algorithm with optimizations.

In [ ]:
skp = Perceptron(max_iter=1000, eta0=0.1, tol=1e-3)
skp.fit(X_train, y_train)
sk_preds = skp.predict(X_test)
print("Accuracy:", accuracy_score(y_test, sk_preds))

## 3. Logistic regression from scratch

Logistic regression is a linear model for binary classification. It predicts the probability that input $x$ belongs to class 1 using the sigmoid function:

$$
p(y=1|x) = \sigma(w^T x + b) = \frac{1}{1 + e^{-(w^T x + b)}}
$$

where $w$ is the weight vector, $b$ is the bias, and $\sigma(z)$ is the sigmoid.

**Training steps:**

1. Initialize $w$ and $b$.
2. Compute predicted probabilities: $\hat{y} = \sigma(w^T x + b)$.
3. Compute binary cross-entropy loss:

    $$
    L = -\frac{1}{n} \sum_{i=1}^n \left[ y_i \log \hat{y}_i + (1-y_i) \log (1-\hat{y}_i) \right]
    $$

4. Compute gradients:

    $$
    \frac{\partial L}{\partial w} = \frac{1}{n} X^T (\hat{y} - y), \quad
    \frac{\partial L}{\partial b} = \frac{1}{n} \sum_{i=1}^n (\hat{y}_i - y_i)
    $$

5. Update $w$ and $b$ with gradient descent.
6. Repeat until convergence.

Classify as 1 if $p(y=1|x) \geq 0.5$, else 0.


In [ ]:
X, y = load_iris(return_X_y=True)
mask = y < 2
X = X[mask, :2]
y = y[mask]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
w = np.zeros(X_train.shape[1])
b = 0.
num_iterations = 200
learning_rate = 0.1

In [ ]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [ ]:
for _ in range(num_iterations):
    z = X_train @ w + b
    preds = sigmoid(z)
    grad_w = X_train.T @ (preds - y_train) / len(y_train)
    grad_b = np.mean(preds - y_train)
    w -= learning_rate * grad_w
    b -= learning_rate * grad_b

preds = (sigmoid(X_test @ w + b) >= 0.5).astype(int)
print("Accuracy:", accuracy_score(y_test, preds))

### scikit-learn logistic regression

scikit-learn's `LogisticRegression` uses regularization and a reliable solver to fit the same model.

In [ ]:
clf = LogisticRegression(max_iter=200)
clf.fit(X_train, y_train)
sk_preds = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, sk_preds))